In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import metrics
%matplotlib inline

In [62]:
df = pd.read_csv(r"data/final_data.csv")
df.head()

,Date,State,Outage,Number of Customers Affected,Num Counties Affected,Season,AWND,PRCP,SNOW,SNWD,...,TMIN,Fog,Thunder,Hail,Dust,Tornado,Wind,Rain,Snow,hours
0,2017-01-01,Alabama,0,0.0,0.0,Winter,3.1875,57.9625,0.0,0.0,...,12.8375,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-02,Alabama,0,0.0,0.0,Winter,4.0625,72.2625,0.0,0.0,...,15.9625,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-03,Alabama,0,0.0,0.0,Winter,2.6875,0.3750,0.0,0.0,...,12.0000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-04,Alabama,0,0.0,0.0,Winter,4.0375,0.0000,0.0,0.0,...,4.0375,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-05,Alabama,0,0.0,0.0,Winter,2.1625,0.1250,0.0,0.0,...,1.4250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Exploring the data features

In [ ]:
df.shape

In [ ]:
#function to calculate descriptive statistics of numerical data
def stats(data):
    missing = np.count_nonzero(np.isnan(np.array(data,dtype='float64')))
    data = data[~pd.isnull(data)]
    median = round(np.median(data),2)
    std = round(np.std(data),2)
    mean = round(np.mean(data),2)
    minumum = round(np.min(data),2)
    maximum = round(np.max(data),2)
    return missing, mean, median, std, minumum, maximum

#function to calculate descriptive statistics of categorical data
def stats_categorial(data):
    missing = sum(pd.isnull(data))
    data = data[~pd.isnull(data)]
    values, counts = np.unique(data, return_counts=True)
    s = ''
    for i in range(len(values)):
        s = s+values[i]+'('+str(counts[i])+')'
    return missing, len(values), s

#function to check whether a column in a dataframe is numeric
def isnumeric(data):
    try:
        [float(i) for i in data]
        return True
    except ValueError:
        return False  

In [ ]:
# function to print the descriptive statistics of a given dataframe
def brief(df):
    df1_array=df.values
    column_names=list(df.columns.values)
    column_detail_numeric = []
    column_detail_categorical = []
    index = []
    cat_index = []
    for i in range(df1_array.shape[1]):
        index.append(i+1)
        if(isnumeric(df1_array[:,i])):
            missing, mean, median, std, minumum, maximum = stats(df1_array[:,i])
            column=column_names[i]
            att_id=(column_names).index(column_names[i])+1
            column_detail_numeric.append([att_id,column,missing,mean,median,std,minumum,maximum])
        else:
            cat_index.append(i)
    for i in cat_index:
        missing, arity, counts = stats_categorial(df1_array[:,i])
        column=column_names[i]
        att_id=(column_names).index(column_names[i])+1
        column_detail_categorical.append([att_id,column,missing,arity,counts])
    
    header= ['Attribute_Id', 'Attribute_Name','Missing','Mean','Median','Sdev','Min','Max']
    index = index[:len(column_detail_numeric)]
    print("real valued attributes")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(pd.DataFrame(column_detail_numeric,index,header))  
    header_categorical=['Attribute_Id', 'Attribute_Name','Missing','arity','MCV_Counts']
    index_categorical=index[:len(column_detail_categorical)]
    print("symbolic attributes")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(pd.DataFrame(column_detail_categorical,index_categorical,header_categorical))      
    print("\n")

In [ ]:
brief(df)  

In [63]:
no_counties_affected = df[df['Num Counties Affected'].isnull()]
no_counties_affected

,Date,State,Outage,Number of Customers Affected,Num Counties Affected,Season,AWND,PRCP,SNOW,SNWD,...,TMIN,Fog,Thunder,Hail,Dust,Tornado,Wind,Rain,Snow,hours
21,2017-01-22,Alabama,1,29965.0,NaN,Winter,4.325000,26.787500,0.0,0.0,...,11.175000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,43.433333
22,2017-01-23,Alabama,1,29965.0,NaN,Winter,5.975000,0.462500,0.0,0.0,...,9.162500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.433333
92,2017-04-03,Alabama,1,86330.0,NaN,Spring,4.662500,50.862500,0.0,0.0,...,15.712500,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,31.000000
123,2017-05-04,Alabama,1,60377.0,NaN,Spring,4.912500,23.175000,0.0,0.0,...,11.187500,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27.000000
165,2017-06-15,Alabama,1,82713.0,NaN,Summer,2.112500,13.175000,0.0,0.0,...,20.762500,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,39.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78545,2017-06-11,Wisconsin,1,53610.0,NaN,Summer,5.166667,0.000000,0.0,0.0,...,22.766667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.666667
79314,2019-07-20,Wisconsin,1,50000.0,NaN,Summer,4.500000,30.733333,0.0,0.0,...,19.233333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.666667
79315,2019-07-21,Wisconsin,1,50000.0,NaN,Summer,2.900000,0.266667,0.0,0.0,...,17.600000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.666667
79316,2019-07-22,Wisconsin,1,50000.0,NaN,Summer,3.900000,0.000000,0.0,0.0,...,15.933333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.666667


In [64]:
no_counties_affected['Outage'].unique()

array([1])

In [65]:
df['Rain'].unique()

array([0.])

In [66]:
df = df.drop(columns = ['Date', 'Num Counties Affected', 'Rain'], axis=1)
df.head()

,State,Outage,Number of Customers Affected,Season,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Fog,Thunder,Hail,Dust,Tornado,Wind,Snow,hours
0,Alabama,0,0.0,Winter,3.1875,57.9625,0.0,0.0,19.712500,12.8375,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alabama,0,0.0,Winter,4.0625,72.2625,0.0,0.0,20.537500,15.9625,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alabama,0,0.0,Winter,2.6875,0.3750,0.0,0.0,20.071429,12.0000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Alabama,0,0.0,Winter,4.0375,0.0000,0.0,0.0,15.075000,4.0375,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Alabama,0,0.0,Winter,2.1625,0.1250,0.0,0.0,14.387500,1.4250,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
df.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

## Handling missing data

In [68]:
df.isnull().mean().sort_values(ascending=True)

State                           0.000000
Wind                            0.000000
Tornado                         0.000000
Dust                            0.000000
Hail                            0.000000
Thunder                         0.000000
Fog                             0.000000
TMIN                            0.000000
TMAX                            0.000000
PRCP                            0.000000
Season                          0.000000
Outage                          0.000000
Snow                            0.000000
hours                           0.000000
AWND                            0.000576
Number of Customers Affected    0.000919
SNOW                            0.019743
SNWD                            0.040098
dtype: float64

In [69]:
df = df.dropna()
df.isnull().mean().sort_values(ascending=True)

State                           0.0
Wind                            0.0
Tornado                         0.0
Dust                            0.0
Hail                            0.0
Thunder                         0.0
Fog                             0.0
TMIN                            0.0
TMAX                            0.0
SNWD                            0.0
SNOW                            0.0
PRCP                            0.0
AWND                            0.0
Season                          0.0
Number of Customers Affected    0.0
Outage                          0.0
Snow                            0.0
hours                           0.0
dtype: float64

Dropping the data since missingness is less than 1%

In [71]:
df.State.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

## Treating categorical value

In [72]:
#label encoding for State
le = LabelEncoder()
df['State'] = le.fit_transform(df['State'])

#onehot encoding for Season
df = pd.concat([df, pd.get_dummies(df['Season'], prefix='Season')], axis=1)
df = df.drop(columns=['Season'], axis=1)
df.head()

,State,Outage,Number of Customers Affected,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Fog,...,Hail,Dust,Tornado,Wind,Snow,hours,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,0,0,0.0,3.1875,57.9625,0.0,0.0,19.712500,12.8375,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
1,0,0,0.0,4.0625,72.2625,0.0,0.0,20.537500,15.9625,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
2,0,0,0.0,2.6875,0.3750,0.0,0.0,20.071429,12.0000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
3,0,0,0.0,4.0375,0.0000,0.0,0.0,15.075000,4.0375,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1
4,0,0,0.0,2.1625,0.1250,0.0,0.0,14.387500,1.4250,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1


## Finding corelation with target variable

In [ ]:
corr = df.corr()["Number of Customers Affected"].reset_index().sort_values(by="Number of Customers Affected",ascending=False)
corr

In [ ]:
#correlation heatmap of dataset
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(20, 20))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = colormap,
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

In [ ]:
correlation_heatmap(df)

In [ ]:
df_main = df.copy()
df_main.head(2)

## Random forest feature selection

In [ ]:
X = df_main.drop(["Number of Customers Affected"], axis=1)
y = df_main["Number of Customers Affected"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
feature_names = [f'feature {i}' for i in range(X.shape[1])]
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
dict(reversed(sorted(zip(rf.feature_importances_, X.columns.values))))

In [ ]:
features = X.columns
importances = rf.feature_importances_
indices = np.argsort(importances)[-9:] 
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

## Normalizing the data

In [73]:
X = df.drop(['Number of Customers Affected'], axis=1)
y = df['Number of Customers Affected']

In [74]:
scaler = StandardScaler()

In [84]:
mean_std_dict = {}
for col in df.columns:
    mean_std_dict[col] = [df[col].mean(), df[col].std()]

In [85]:
mean_std_dict

{'State': [23.487577957264083, 13.852090218156427],
 'Outage': [0.01435180451896534, 0.1189370043776488],
 'Number of Customers Affected': [3109.0457136284635, 48243.95196142869],
 'AWND': [3.6046979067938993, 1.4829674266885424],
 'PRCP': [2.6506215026527857, 6.174512701716785],
 'SNOW': [2.154551489298647, 13.406634232864967],
 'SNWD': [17.32275710760568, 72.79354447532016],
 'TMAX': [18.445801819436678, 11.322569065874935],
 'TMIN': [6.772201132654679, 10.388184118762583],
 'Fog': [0.6803496574992333, 0.4663440579303284],
 'Thunder': [0.24240875166138431, 0.4285429917670362],
 'Hail': [0.010620079746447193, 0.10250574586962279],
 'Dust': [0.0050991718638176056, 0.07122664631620647],
 'Tornado': [0.000830692158266026, 0.02880994127495925],
 'Wind': [0.00011501891422144975, 0.010724138876776201],
 'Snow': [2.5559758715877723e-05, 0.005055633695301739],
 'hours': [1.2550957638702203, 13.112778029383072],
 'Season_Fall': [0.2230983539515387, 0.4163264266446045],
 'Season_Spring': [0.282

In [76]:
df_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [108]:
df_scaled.Outage.unique()

array([-0.12066805,  8.28719791])

In [106]:
def normalize_data(df):
    mean_std_dict = {'State': [23.487577957264083, 13.852090218156427],
                    'Outage': [0.01435180451896534, 0.1189370043776488],
                    'AWND': [3.6046979067938993, 1.4829674266885424],
                    'PRCP': [2.6506215026527857, 6.174512701716785],
                    'SNOW': [2.154551489298647, 13.406634232864967],
                    'SNWD': [17.32275710760568, 72.79354447532016],
                    'TMAX': [18.445801819436678, 11.322569065874935],
                    'TMIN': [6.772201132654679, 10.388184118762583],
                    'Fog': [0.6803496574992333, 0.4663440579303284],
                    'Thunder': [0.24240875166138431, 0.4285429917670362],
                    'Hail': [0.010620079746447193, 0.10250574586962279],
                    'Dust': [0.0050991718638176056, 0.07122664631620647],
                    'Tornado': [0.000830692158266026, 0.02880994127495925],
                    'Wind': [0.00011501891422144975, 0.010724138876776201],
                    'Snow': [2.5559758715877723e-05, 0.005055633695301739],
                    'hours': [1.2550957638702203, 13.112778029383072],
                    'Season_Fall': [0.2230983539515387, 0.4163264266446045],
                    'Season_Spring': [0.28217973622329007, 0.4500632414908624],
                    'Season_Summer': [0.23780799509252631, 0.42574366585275464],
                    'Season_Winter': [0.2569139147326449, 0.43693431425849816]}
    for col in df.columns:
        df[col] = df[col].apply(lambda x: (x - mean_std_dict[col][0])/mean_std_dict[col][1])
    return df

asdf = normalize_data(X[0:1].copy())
asdf

,State,Outage,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Fog,Thunder,Hail,Dust,Tornado,Wind,Snow,hours,Season_Fall,Season_Spring,Season_Summer,Season_Winter
0,-1.695598,-0.120667,-0.281326,8.958096,-0.160708,-0.237971,0.111874,0.583865,0.685439,1.76783,-0.103605,-0.071591,-0.028834,-0.010725,-0.005056,-0.095715,-0.535874,-0.626978,-0.558571,1.700681


## Building models

## RandomForestRegressor

In [ ]:
X = df_scaled
y = df['Number of Customers Affected']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

In [ ]:
rf_y_pred = rf_model.predict(X_test)

In [ ]:
print('Root Mean Squared Error for Random forest regressor is:', np.sqrt(metrics.mean_squared_error(y_test, rf_y_pred)))

In [ ]:
print('R-squared for Random forest regressor is:', metrics.r2_score(y_test, rf_y_pred))

# Xgboost Regressor

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
#XGBoost hyper-parameter tuning
param_tuning = {
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 6, 8],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

In [ ]:
xgb_model = XGBRegressor()

In [ ]:
gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,
                           scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

In [ ]:
gsearch.fit(X_train,y_train)

In [ ]:
gsearch.best_params_

In [ ]:
xgb_model = XGBRegressor(objective = 'reg:squarederror',
                        learning_rate = 0.1,
                        max_depth = 5,
                        n_estimators = 500, 
                        early_stopping_rounds=5)

In [ ]:
xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

In [121]:
y_pred_xgb = xgb_model.predict(X_test)
print('Root Mean Squared Error for Xgboost regressor is:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_xgb)))
print('R-squared for Xgboost regressor is:', metrics.r2_score(y_test, y_pred_xgb))

Root Mean Squared Error for Xgboost regressor is: 12078.955476305218
R-squared for Xgboost regressor is: 0.9361181898136481


In [ ]:
from joblib import dump
dump(xgb_model, 'reg_model.joblib')